In [100]:
import torch
from torch import nn
import torch.nn.functional as F
from datasets import load_dataset
import fastcore.all as fc
import matplotlib.pyplot as plt
import matplotlib as mpl
import torchvision.transforms.functional as TF
from torch.utils.data import default_collate, DataLoader
import torch.optim as optim
import pickle


In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [2, 2]

In [101]:
dataset_nm = 'mnist'
x,y = 'image', 'label'
ds = load_dataset(dataset_nm)

Found cached dataset mnist (/Users/arun/.cache/huggingface/datasets/mnist/mnist/1.0.0/9d494b7f466d6931c64fb39d58bb1249a4d85c9eb9865d9bc20960b999e2a332)
100%|██████████| 2/2 [00:00<00:00, 35.54it/s]


In [112]:
def transform_ds(b):
    b[x] = [TF.to_tensor(ele) for ele in b[x]]
    return b

In [ ]:
dst = ds.with_transform(transform_ds)
plt.imshow(dst['train'][0]['image'].permute(1,2,0));

In [103]:
bs = 1024
class DataLoaders:
    def __init__(self, train_ds, valid_ds, bs, collate_fn, **kwargs):
        self.train = DataLoader(train_ds, batch_size=bs, shuffle=True, collate_fn=collate_fn, **kwargs)
        self.valid = DataLoader(valid_ds, batch_size=bs*2, shuffle=False, collate_fn=collate_fn, **kwargs)

def collate_fn(b):
    collate = default_collate(b)
    return (collate[x], collate[y])

(torch.Size([1024, 1, 28, 28]), torch.Size([1024]))

In [ ]:
dls = DataLoaders(dst['train'], dst['test'], bs=bs, collate_fn=collate_fn)
xb,yb = next(iter(dls.train))
xb.shape, yb.shape

In [105]:
class Reshape(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
    
    def forward(self, x):
        return x.reshape(self.dim)

In [106]:
def conv(ni, nf, ks=3, s=2, act=nn.ReLU, norm=None):
    layers = [nn.Conv2d(ni, nf, kernel_size=ks, stride=s, padding=ks//2)]
    if norm:
        layers.append(norm)
    if act:
        layers.append(act())
    return nn.Sequential(*layers)

def _conv_block(ni, nf, ks=3, s=2, act=nn.ReLU, norm=None):
    return nn.Sequential(
        conv(ni, nf, ks=ks, s=1, norm=norm, act=act),
        conv(nf, nf, ks=ks, s=s, norm=norm, act=act),
    )

class ResBlock(nn.Module):
    def __init__(self, ni, nf, s=2, ks=3, act=nn.ReLU, norm=None):
        super().__init__()
        self.convs = _conv_block(ni, nf, s=s, ks=ks, act=act, norm=norm)
        self.idconv = fc.noop if ni==nf else conv(ni, nf, ks=1, s=1, act=None)
        self.pool = fc.noop if s==1 else nn.AvgPool2d(2, ceil_mode=True)
        self.act = act()
    
    def forward(self, x):
        return self.act(self.convs(x) + self.idconv(self.pool(x)))

In [107]:
def cnn_classifier():
    return nn.Sequential(
        ResBlock(1, 8, norm=nn.BatchNorm2d(8)),
        ResBlock(8, 16, norm=nn.BatchNorm2d(16)),
        ResBlock(16, 32, norm=nn.BatchNorm2d(32)),
        ResBlock(32, 64, norm=nn.BatchNorm2d(64)),
        ResBlock(64, 64, norm=nn.BatchNorm2d(64)),
        conv(64, 10, act=False),
        nn.Flatten(),
    )

In [108]:
def kaiming_init(m):
    if isinstance(m, (nn.Conv1d, nn.Conv2d, nn.Conv3d)):
        nn.init.kaiming_normal_(m.weight)        

In [195]:
model = cnn_classifier()
model.apply(kaiming_init)
lr = 0.1
max_lr = 0.3
epochs = 5
opt = optim.AdamW(model.parameters(), lr=lr)
sched = optim.lr_scheduler.OneCycleLR(opt, max_lr, total_steps=len(dls.train), epochs=epochs)
for epoch in range(epochs):
    for train in (True, False):
        accuracy = 0
        dl = dls.train if train else dls.valid
        for xb,yb in dl:
            preds = model(xb)
            loss = F.cross_entropy(preds, yb)
            if train:
                loss.backward()
                opt.step()
                opt.zero_grad()
            with torch.no_grad():
                accuracy += (preds.argmax(1).detach().cpu() == yb).float().mean()
        if train:
            sched.step()
        accuracy /= len(dl)
        print(f"{'train' if train else 'eval'}, epoch:{epoch+1}, loss: {loss.item():.4f}, accuracy: {accuracy:.4f}")

train, epoch:1, loss: 0.0776, accuracy: 0.9172
eval, epoch:1, loss: 0.0372, accuracy: 0.9818
train, epoch:2, loss: 0.0571, accuracy: 0.9828
eval, epoch:2, loss: 0.0287, accuracy: 0.9863
train, epoch:3, loss: 0.0425, accuracy: 0.9847
eval, epoch:3, loss: 0.0256, accuracy: 0.9865
train, epoch:4, loss: 0.0271, accuracy: 0.9868
eval, epoch:4, loss: 0.0378, accuracy: 0.9826
train, epoch:5, loss: 0.0395, accuracy: 0.9844
eval, epoch:5, loss: 0.0307, accuracy: 0.9873


In [196]:
torch.save(model.state_dict(), 'classifier.pth')

In [197]:
loaded_model = cnn_classifier()
loaded_model.load_state_dict(torch.load('classifier.pth'))
loaded_model.eval();

In [206]:
def predict(img):
    with torch.no_grad():
        img = img[None,]
        pred = loaded_model(img)[0]
        pred_probs = F.softmax(pred, dim=0)
        pred = [{"digit": i, "prob": f'{prob*100:.2f}%', 'logits': pred[i]} for i, prob in enumerate(pred_probs)]
        pred = sorted(pred, key=lambda ele: ele['digit'], reverse=False)
    return pred

In [204]:
img = xb[0].reshape(1, 28, 28)
print(yb[0])
predict(img)

tensor(5)


[{'digit': 0, 'prob': '21.42%', 'logits': tensor(0.0559)},
 {'digit': 8, 'prob': '19.44%', 'logits': tensor(-0.0408)},
 {'digit': 4, 'prob': '18.08%', 'logits': tensor(-0.1135)},
 {'digit': 9, 'prob': '16.41%', 'logits': tensor(-0.2104)},
 {'digit': 6, 'prob': '12.23%', 'logits': tensor(-0.5049)},
 {'digit': 1, 'prob': '6.87%', 'logits': tensor(-1.0806)},
 {'digit': 7, 'prob': '2.33%', 'logits': tensor(-2.1633)},
 {'digit': 5, 'prob': '1.19%', 'logits': tensor(-2.8386)},
 {'digit': 2, 'prob': '1.06%', 'logits': tensor(-2.9527)},
 {'digit': 3, 'prob': '0.97%', 'logits': tensor(-3.0359)}]

#### commit to .py file for deployment

In [207]:
!jupyter nbconvert --to script --TagRemovePreprocessor.remove_cell_tags="exclude" --TemplateExporter.exclude_input_prompt=True mnist_classifier.ipynb


[NbConvertApp] Converting notebook mnist_classifier.ipynb to script
[NbConvertApp] Writing 2905 bytes to mnist_classifier.py
